## solving QP primal

In [ ]:
using Random
using MathOptInterface
using Dualization
using OSQP
using LinearAlgebra

const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities;

In [ ]:
n = 2 # variable dimension
m = 6; # no of inequality constraints

In [ ]:
# using example on https://osqp.org/docs/examples/setup-and-solve.html
Q = [4. 1.;1. 2.]
q = [1.; 1.]
G = [1. 1.;
    1. 0.;
    0. 1.;
    -1. -1.;
    -1. 0.;
    0. -1.]
h = [1.;
    0.7;
    0.7; 
    -1.;
    0.;
    0.];

In [ ]:
model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
x = MOI.add_variables(model, n);

In [ ]:
# define objective

quad_terms = MOI.ScalarQuadraticTerm{Float64}[]
for i in 1:n
    for j in i:n # indexes (i,j), (j,i) will be mirrored. specify only one kind
        push!(quad_terms, MOI.ScalarQuadraticTerm(Q[i,j],x[i],x[j]))
    end
end

objective_function = MOI.ScalarQuadraticFunction(MOI.ScalarAffineTerm.(q, x),quad_terms,0.)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

In [ ]:
# maintain constrain to index map - will be useful later
constraint_map = Dict()

# add constraints
for i in 1:m
    ci = MOI.add_constraint(model,MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(G[i,:], x), 0.),MOI.LessThan(h[i]))
    constraint_map[ci] = i
end

In [ ]:
MOI.optimize!(model)

In [ ]:
@assert MOI.get(model, MOI.TerminationStatus()) in [MOI.LOCALLY_SOLVED, MOI.OPTIMAL]

In [ ]:
x̄ = MOI.get(model, MOI.VariablePrimal(), x);

In [ ]:
# objective value (predicted vs actual) sanity check
@assert norm(x̄ - [0.3; 0.7]) < 1e-3

## find and solve dual problem 

ideally the primal, dual of QP are as follows

| primal | dual |
|--------|------|
$$\text{min } \frac{1}{2}x^TQx + q^Tx$$  | $$\text{max } -\frac{1}{2}y^TQ^{-1}y - u^Th$$
$$\text{s.t.  }Gx <= h$$ | $$\text{s.t.  } u \geq 0, u \in R^m$$
    $$ $$ |    $$y = q + G^Tu$$
  
- Each primal variable becomes a dual constraint
- Each primal constraint becomes a dual variable

But refer https://www.juliaopt.org/Dualization.jl/dev/manual/, it involves inverse/psuedoinverse matrices (Q^-1). 
To avoid that `Dualization.jl` introduces a slack variable `w`. Put `y = Qw`, hence we have

| primal | dual |
|--------|------|
$$\text{min } \frac{1}{2}x^TQx + q^Tx$$  | $$\text{max } -\frac{1}{2}w^T Q w - u^Th$$
$$\text{s.t.  }Gx <= h$$ | $$\text{s.t.  } u \geq 0, u \in R^m$$
   $$ $$  |    $$Qw = q + G^Tu$$

In [ ]:
joint_object    = dualize(model)
dual_model_like = joint_object.dual_model # this is MOI.ModelLike, not an MOI.AbstractOptimizer; can't call optimizer on it
primal_dual_map = joint_object.primal_dual_map;

In [ ]:
# copy the dual model objective, constraints, and variables to an optimizer
dual_model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
MOI.copy_to(dual_model, dual_model_like)

# solve dual
MOI.optimize!(dual_model);

In [ ]:
# check if strong duality holds
@assert abs(MOI.get(model, MOI.ObjectiveValue()) - MOI.get(dual_model, MOI.ObjectiveValue())) <= 1e-3

## derive and verify KKT conditions

In [ ]:
is_equality(set::S) where {S<:MOI.AbstractSet} = false
is_equality(set::MOI.EqualTo{T}) where T = true

map = primal_dual_map.primal_con_dual_var;

**complimentary slackness**: $$\mu_{i}(G\bar x -h)_i=0 \qquad \text{ where } i=1..m$$

**dual feasibility**
$$ \mu_i \leq 0$$

In [ ]:
for con_index in keys(map)
    μ = MOI.get(dual_model, MOI.VariablePrimal(), map[con_index][1])
    i = constraint_map[con_index]
    
    # (Gx - h)[i] = 0
    @assert G[i,:]'*x̄ - h[i] < 1e-3

    # μ[i] <= 0
    @assert μ <= 1e-3
end

**checking stationarity**
$$ Qx + q = A\mu$$

In [ ]:
for i in 1:n
    G_sum = 0
    
    for con_index in keys(map)
        μ = MOI.get(dual_model, MOI.VariablePrimal(), map[con_index][1])
        j = constraint_map[con_index]
        G_sum += μ*G[j,i]
    end
    
    @assert abs(G_sum - (Q*x̄ + q)[i]) < 1e-2
end